# **DASCore Basics**

December 1, 2024

This notebook Shows how to work with DASCore's [`Patch`](https://dascore.org/api/dascore/core/patch/Patch.html). It is a shortened version of the [DASCore's Patch tutorial](https://dascore.org/tutorial/patch.html). 

<a target="_blank" href="https://colab.research.google.com/github/DASDAE/seg_tutorial/blob/master/02_patch.ipynb">

</a>  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>

#### Useful links: 
* [DASCore Tutorial](https://dascore.org/tutorial/concepts.html)
* [Numpy Dates and Times](https://numpy.org/devdocs/reference/arrays.datetime.html)
* [Pint Units Library](https://pint.readthedocs.io/en/stable/)


In [ ]:
%%capture

# First ensure DASCore is installed. If not, install and restart the kernel.
try:
    import dascore as dc
except ImportError:
    !pip install dascore
    # resetart kernel
    import IPython
    IPython.Application.instance().kernel.do_shutdown(True) #automatically restarts kernel

from rich import print


# Patch
To demonstrate the patch, we use an example seismic event recorded shown in [this study](https://www.frontiersin.org/journals/earth-science/articles/10.3389/feart.2022.907749/full).

The `Patch` is composed of:
- data: the array of measurements
- attrs: the non-coordinate metdata
- dims: a tuple of dimension names (eg 'time', 'distance')
- coords: The coordinates of each dimension (plus others)

It is *immutable* which means that once a patch is created it cannot (easily) be changed. This makes it safe to share compontents between patches and simplifies parallel processing. 

The patch design was inspired by [Xarray's DataArray](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.html).

In [ ]:
# The get_example_patch function is useful for loading example/test patches.
patch = dc.get_example_patch("example_event_1")

In [ ]:
# The patch str rep. provides a summary
print(patch)

## Patch Components: Data and Dims

The data array can be accessed with `data` and updated with (`Patch.update`)[https://dascore.org/tutorial/patch.html#update]

In [ ]:
# Get the data array
array = patch.data

In [ ]:
# Divide the data by 1_000_0000 and create a new patch.
patch_new_data = patch.update(data=array / 1_000_000)

The dimensions are accessed via the `dims` attribute.

In [ ]:
print(patch.dims)

### **Exercise** (Patch Data)
Calculate the and print following:

1. The number of samples in the data
2. The minimum and maximum value of the data

## Patch Components: Attrs
Attrs contain non-dimensional metdata. Acquisition/interrogator identifiers, custom tags, etc.


In [ ]:
print(patch.attrs)

Notice the data units, `acquisition_id`, and `cable_id` are not set. We could set them like so:

In [ ]:
patch_updated_attrs = patch.update_attrs(
    data_units="nanostrain/s", 
    acquisition_id="experiment_12",
    cable_id="b202393ad",
)
print(patch_updated_attrs)

Note: [Patch.set_units](https://dascore.org/api/dascore/proc/units/set_units.html) and [Patch.convert_units](https://dascore.org/api/dascore/proc/units/convert_units.html) are preferrible for handling data unit conversions.


### **Exercise** (Patch Attrs)
Calculate the and print following:

1. The number of samples in the data
2. The minimum and maximum value of the data

## Patch Components: Coords

`Coords` represent information about the coordinates (not coordinate systems) associated with a patch. These include, but arent limited to, the dimensions as well as others.


In [ ]:
print(patch.coords) # The coordinates and their labels. 

Coordinate objects can be accessed with [`Patch.get_coord`](https://dascore.org/api/dascore/proc/coords/get_coord.html). 

In [ ]:
dist_coord = patch.get_coord("distance")

# start, stop, step, units, values
dist_start = dist_coord.min()
dist_stop = dist_coord.max()
dist_step = dist_coord.step
dist_units = dist_coord.units
dist_array = dist_coord.values

Alternatively, [`Patch.get_array`](https://dascore.org/api/dascore/proc/coords/get_array.html) simply returns the numpy array of the coordinate. 

In [ ]:
# The values of the distance dimension
dist = patch.get_array("distance")

# The value of the time dimension in seconds
time_s = to_float(patch.get_array("distance"))

Coordinates can be renamed using [`Patch.rename_coord`](https://dascore.org/api/dascore/proc/coords/rename_coords.html)

In [ ]:
patch

In [ ]:
patch_renamed_coords = patch.rename_coords(distance="depth")
print(patch_renamed_coords.dims)

Or updated using [`Patch.update_coords`](https://dascore.org/api/dascore/proc/coords/update_coords.html)

In [ ]:
from dascore.units import m

dist = patch.get_array("distance")
patch_new_dist = patch.update_coords(distance=(dist + 12)*m)

dist2 = patch_new_dist.get_coord("distance")

In [ ]:
print(dist2)

### **Exercise** (Patch Coords)

Calculate the and print following parameters:

1. The duration (time) of the patch recording

2. Reset the start of the time coordiante to noon on the release date of Elton John's single Rocket Man (17 April 1972)


## Visualization
DASCore provides a few visualization in the `patch.viz` namespace.


In [ ]:
# The classic waterfall plot
patch.viz.waterfall(scale=0.2);

In [ ]:
# A wiggle plot
patch.viz.wiggle(scale=0.2)

### **Exercise** (Patch Viz)
Redo the waterfall plot, but play with the `scale` parameter to make the event more visible.

## Trimming and sub-selection
[`Patch.select`]() is used to trim patches. For example, to zoom in on the down-going reflection.

In [ ]:
trimmed = patch.select(time=(.16, .22), distance =(600,800))

In [ ]:
trimmed.viz.waterfall(scale=0.1);

Trimming can also be done via samples

In [ ]:
# Remove 20 samples from start and end of time dimension using samples=True
trimmed_patch = patch.select(time=(20, -20), samples=True)

In [ ]:
# Remove .05 seconds from start and end of patch using relative=True
trimmed_patch = patch.select(time=(0.05, -0.05), relative=True)

### **Exercise** (Patch Select)
Remove the first and last 10 spatial channels.

## Patch Processing
DASCore provides a variety of [processing](https://dascore.org/api/dascore/proc.html) patch methods. This section will highlight a few of these.


### Pass Filtering
The `Patch.pass_filter` method is used to apply bandpass, lowpass, and highpass filters to the data along a specified dimension.

In [ ]:
patch_bp = patch.pass_filter(time=(100, 300))  # apply a 100Hz to 300Hz highpass
patch_lp = patch.pass_filter(time=(None, 300))  # apply a 300Hz lowpass
patch_bp = patch.pass_filter(time=(50, None))  # apply a 50Hz highpass

### **Exercise** (Patch Processing 1)
Plot each of the filtered patches above. Which filtering technique did the best at accentuating the event signal? 

### Detrend

`Patch.detrend` applies a linear detrend along a specified dimension (axis).

### Decimate
`decimate` reduces the number of samples in the specified dimension. By default, a lowpass filter is applied to avoid aliasing.

In [ ]:
patch_dec = patch.decimate(time=10)  # keep every 10th sample along time axis

### Resample

Like `decimate`, `resample` is used to change the sampling rate of the patch along a specific dimension. Unlike `decimate`, however, non-integar multiples can be used.

In [ ]:
patch_resamp = patch.resample(distance=15)  # change spatial sampling to 15m

### **Exercise** (Processing 2)

Using the processing methods above, or others you find in the [processing module documentation](https://dascore.org/api/dascore/proc.html), apply processing to the example patch to accentuate the primary phases of the event and plot it.

## Patch Transformations
DASCore includes several [transformation functions](https://dascore.org/api/dascore/transform.html) that can be used on patches. This section shows some examples.

In [ ]:
# Load a cleaned-up patch from the previous example.
patch = dc.get_example_patch("example_event_2")

In [ ]:
# Perform an rft along the time axis. 
patch_fft = patch.dft("time", real=True)

In [ ]:
# Integrate along time domain
patch_int = patch.integrate("distance")

### **Exercise** (Transformation)

Perform an RFT in the time domain and `Patch.taper_range` to notch out frequencies between 58 and 62 Hz and plot.